In [2]:
import requests
import pandas as pd
import json
import numpy as np
from pymongo import MongoClient
from sshtunnel import SSHTunnelForwarder
import threading
import uuid
import time
import random

### url格式

### 主要調整
- count 該頁面顯示的商品數量
- keyword 查詢商品名稱
- offset 第二頁的起始頁碼(依據前一頁查詢的商品數量決定)

In [ ]:
url = 'https://redsky.target.com/v2/plp/search/?channel=web&count=96&keyword={kw}&offset={number}&pricing_store_id=3991&key=ff457966e64d5e877fdbad070f276d18ecec4a01'

### 查詢

### count : 固定為 96
### 查詢商品 : cheese
### oddset : 第一頁為0

In [ ]:
kw = 'cheese'
number = 0
url = 'https://redsky.target.com/v2/plp/search/?channel=web&count=96&keyword={kw}&offset={number}&pricing_store_id=3991&key=ff457966e64d5e877fdbad070f276d18ecec4a01'
data =  requests.get(url.format(kw=kw,number=number)).json()

### 資料完整樣式

In [ ]:
data

{'search_recommendations': {'query': 'cheese'},
 'search_response': {'items': {'Item': [{'subscribable': False,
     'package_dimensions': {'weight': '0.5',
      'weight_unit_of_measure': 'POUND',
      'width': '6.0',
      'depth': '1.5',
      'height': '8.5',
      'dimension_unit_of_measure': 'INCH'},
     'title': 'Shredded Mexican-Style Cheese - 8oz - Good &#38; Gather&#8482;',
     'tcin': '54337113',
     'type': 'Stand Alone',
     'dpci': '284-11-1308',
     'upc': '085239038475',
     'url': '/p/shredded-mexican-style-cheese-8oz-good-38-gather-8482/-/A-54337113',
     'description': "Whether you're putting together a condiments bar for taco Tuesday, adding a finishing touch to a plate of deluxe nachos or simply spreading a little extra flavor over a frozen pizza, you'll love adding Shredded Mexican-Style Cheese from Good & Gather™ to your refrigerator. This reclosable package of Mexican-style cheese contains approximately two cups of shredded cheese, giving you a convenien

### 商品名稱 'title'

### 商品圖片網址 是由images中的'base_url'及 'content_labels'中的'image_url'組成

### 商品價錢 'formatted_current_price'

### 商品文宣 'description'

### 商品評價 'average_rating'

### 商品評價分布 'rating_distribution' 會分別顯示'RatingValue'1-5的次數

### 商品3項指標得分數 'secondary_ratings_averages'包含 'Value', 'Quality', 'Taste'。分數是取裡面的'AverageRating'。 有些不是食物的商品不會有此指標

### 商品瀏覽數 'total_reviews'

### 商品評論 'top_reviews'，裡面又包含'user_nickname','title','rating','review_text'

### 商品描述 'bullet_description' 包含 'Contains','Form','State of Readiness','Package Quantity','Package type','Net weight'

### 商品特色(店家描述) 'soft_bullets'中的'bullets'

### 為了取得商品中的label info ， 先透過json取得商品所在網址，再透過商品_url = 'https://www.target.com'+'url網址' 進入商品頁面，再用request的方式取的label info 的資訊。不是所有商品都有這個

### 爬取一頁

In [ ]:
#商品名稱
commodity_name = [i['title'] for i in data['search_response']['items']['Item']]

#商品圖片網址
commodity_images = [i['images'][0]['base_url']+i['images'][0]['primary'] for i in data['search_response']['items']['Item']]

#商品價錢

commodity_price = [i['price']['formatted_current_price'] for i in data['search_response']['items']['Item']]

#商品url
commodity_url = ['https://www.target.com'+i['url'] for i in data['search_response']['items']['Item']]

#商品文宣
commodity_description = []
for i in data['search_response']['items']['Item']:
    try:
        commodity_description.append(i['description'])
    except KeyError:
        commodity_description.append(np.nan)

#商品評價
commodity_average_rating = [i['average_rating'] for i in data['search_response']['items']['Item']]


#商品評價(次數)分布
#1星
commodity_RatingValue1 = [i['rating_distribution'][0]['Count'] for i in data['search_response']['items']['Item']]

#2星
commodity_RatingValue2 = [i['rating_distribution'][1]['Count'] for i in data['search_response']['items']['Item']]

#3星
commodity_RatingValue3 = [i['rating_distribution'][2]['Count'] for i in data['search_response']['items']['Item']]

#4星
commodity_RatingValue4 = [i['rating_distribution'][3]['Count'] for i in data['search_response']['items']['Item']]

#5星
commodity_RatingValue5 = [i['rating_distribution'][4]['Count'] for i in data['search_response']['items']['Item']]

#商品3項指標得分數  有些商品會沒有這些資料，應該移除
# Value
commodity_Value_score = []
for i in data['search_response']['items']['Item']:
    try:
        commodity_Value_score.append(i['secondary_ratings_averages']['Value']['AverageRating'])
    except KeyError:
        commodity_Value_score.append(np.nan)

#Quality
commodity_Quality_score = []
for i in data['search_response']['items']['Item']:
    try:
        commodity_Quality_score.append(i['secondary_ratings_averages']['Quality']['AverageRating'])
    except KeyError:
        commodity_Quality_score.append(np.nan)

#Taste
commodity_Taste_score = []
for i in data['search_response']['items']['Item']:
    try:
        commodity_Taste_score.append(i['secondary_ratings_averages']['Taste']['AverageRating'])
    except KeyError:
        commodity_Taste_score.append(np.nan)

#商品瀏覽數
commodity_total_reviews = [i['total_reviews'] for i in data['search_response']['items']['Item']]

#商品特色(店家描述)
commodity_highlights = []
for i in data['search_response']['items']['Item']:
    try:
        commodity_highlights.append(i['soft_bullets']['bullets'])
    except KeyError:
        commodity_highlights.append(np.nan)

#商品描述 'bullet_description'
commodity_descriptio = [str(i['bullet_description']).replace('<B>','').replace('</B>','').replace("'","") for i in data['search_response']['items']['Item']]

### 裝資料容器

In [3]:
commodity_name = []
commodity_images = []
commodity_price = []
commodity_url = []
commodity_description = []
commodity_average_rating = []
commodity_RatingValue1 = []
commodity_RatingValue2 = []
commodity_RatingValue3 = []
commodity_RatingValue4 = []
commodity_RatingValue5 = []
commodity_Value_score = []
commodity_Quality_score = []
commodity_Taste_score = []
commodity_total_reviews = []
commodity_highlights = []
commodity_descriptio = []

In [2]:
product = pd.read_csv('../data/new_product.csv')
kws = product['大分類'].unique().tolist()

In [13]:
kws

['ADULT INCONTINENCE',
 'ANALGESICS',
 'ANTACIDS',
 'APPLES',
 'AUDIO/VIDEO PRODUCTS',
 'AUTOMOTIVE PRODUCTS',
 'BABY FOODS',
 'BABY HBC',
 'BACON',
 'BAG SNACKS',
 'BAKED BREAD/BUNS/ROLLS',
 'BAKED SWEET GOODS',
 'BAKING MIXES',
 'BAKING NEEDS',
 'BATH TISSUES',
 'BATTERIES',
 'BEEF',
 'BLEACH',
 'BOOKSTORE',
 'BREAD',
 'BREAKFAST SAUSAGE/SANDWICHES',
 'BREAKFAST SWEETS',
 'BROOMS AND MOPS',
 'CAKES',
 'CANDLES/ACCESSORIES',
 'CANDY - PACKAGED',
 'CANNED JUICES',
 'CARROTS',
 'CAT FOOD',
 'CHARCOAL AND LIGHTER FLUID',
 'CHEESE',
 'CHEESES',
 'CHICKEN',
 'CHICKEN/POULTRY',
 'CHRISTMAS  SEASONAL',
 'CIGARETTES',
 'CIGARS',
 'COCOA MIXES',
 'COFFEE FILTERS',
 'COLD AND FLU',
 'COLD CEREAL',
 'CONDIMENTS/SAUCES',
 'CONVENIENT BRKFST/WHLSM SNACKS',
 'COOKIES',
 'COOKIES/CONES',
 'COUPON',
 'CRACKERS/MISC BKD FD',
 'DELI MEATS',
 'DEODORANTS',
 'DIAPERS & DISPOSABLES',
 'DINNER MXS:DRY',
 'DINNER SAUSAGE',
 'DISHWASH DETERGENTS',
 'DOG FOODS',
 'DOMESTIC GOODS',
 'DOMESTIC WINE',
 'DRIED FR

### 爬取多頁

In [134]:
server = SSHTunnelForwarder(('3.113.29.214', 22),
    ssh_password='koxhAS?ZRayMorAddS6*HzZMZ3LFh*wK',
    ssh_username='Administrator',
    remote_bind_address=('127.0.0.1', 27017))

server.start()
client = MongoClient('127.0.0.1', server.local_bind_port)
storedb = client.storedb
collection = storedb.targetadd

kas = ['LUNCHMEAT']
kbs = ['YOGURT' ,  'DELI MEATS' , 'FROZEN PIZZA' , 'PORK' ,'CHEESES' , 'CIGARETTES' , 'TOMATOES' , 'SNACK NUTS' , 'TURKEY' , 'SALAD MIX'
      , 'LUNCHMEAT']
print(len(kas))

kw = ''
kb = ''
stop = 600

number = 0
data_list = []
ax  = 0
while True:
    try:
        url = 'https://redsky.target.com/v2/plp/search/?channel=web&count=96&keyword={kw}&offset={number}&pricing_store_id=3991&key=ff457966e64d5e877fdbad070f276d18ecec4a01'
        data =  requests.get(url.format(kw=kw,number=number)).json()
        for i in data['search_response']['items']['Item']:
            data_dict = {}
            data_dict['commodity_SUB_COMMODITY'] = kb
            data_dict["commodity_name"] = i['title']
            data_dict["commodity_images"] = i['images'][0]['base_url']+i['images'][0]['primary']
            try:
                data_dict['commodity_price'] = i['price']['formatted_current_price'].replace('$' , '')
            except:
                data_dict['commodity_price'] = None

            data_dict['commodity_url'] = 'https://www.target.com'+i['url']
            try:
                data_dict['commodity_description'] = i['description'].replace('<br />' , '')
            except:
                data_dict['commodity_description'] = None 
            data_dict['commodity_total_reviews'] = i['total_reviews']
            data_dict['commodity_descriptio'] = str(i['bullet_description']).replace('<B>','').replace('</B>','').replace("'","")
            data_dict['commodity_average_rating'] = i['average_rating']
            data_list.append(data_dict)
            ax +=  1

        number += 96
        if ax > stop :
            print(len(data_list))
            try:
                collection.insert(data_list)
                print('ok')
            except:
                print(f'{kw}遺漏')
            break
                
    except KeyError: #如果爬取的url 沒有上方的資料，就會產生KeyError，因此break
        print(len(data_list))
        try:
            collection.insert(data_list)
            print('ok')
        except:
            print(f'{kw}遺漏')
        break

        
client.close()
server.stop()

3
667
ok


2021-07-18 23:27:06,773| ERROR   | Socket exception: 遠端主機已強制關閉一個現存的連線。 (10054)


In [39]:
server = SSHTunnelForwarder(('3.113.29.214', 22),
    ssh_password='koxhAS?ZRayMorAddS6*HzZMZ3LFh*wK',
    ssh_username='Administrator',
    remote_bind_address=('127.0.0.1', 27017))

server.start()
client = MongoClient('127.0.0.1', server.local_bind_port)
storedb = client.storedb
collection = storedb.targetdata

def request_target(kws):
    for kw in kws:
        number = 0
        data_list = []
        print(kw)
        while True:
            try:
                url = 'https://redsky.target.com/v2/plp/search/?channel=web&count=96&keyword={kw}&offset={number}&pricing_store_id=3991&key=ff457966e64d5e877fdbad070f276d18ecec4a01'
                data =  requests.get(url.format(kw=kw,number=number)).json()
                for i in data['search_response']['items']['Item']:
                    data_dict = {}
                    data_dict['commodity_SUB_COMMODITY'] = kw
                    data_dict["commodity_name"] = i['title']
                    data_dict["commodity_images"] = i['images'][0]['base_url']+i['images'][0]['primary']
                    try:
                        data_dict['commodity_price'] = i['price']['formatted_current_price'].replace('$' , '')
                    except:
                        data_dict['commodity_price'] = np.nan

                    data_dict['commodity_url'] = 'https://www.target.com'+i['url']
                    try:
                        data_dict['commodity_description'] = i['description'].replace('<br />' , '')
                    except:
                        data_dict['commodity_description'] = np.nan 
                    data_dict['commodity_total_reviews'] = i['total_reviews']
                    data_dict['commodity_descriptio'] = str(i['bullet_description']).replace('<B>','').replace('</B>','').replace("'","")
                    data_dict['commodity_average_rating'] = i['average_rating']
                    data_list.append(data_dict)
                    

                number += 96
                time.sleep(random.uniform(1, 3))
            except KeyError: #如果爬取的url 沒有上方的資料，就會產生KeyError，因此break
                print(len(data_list))
                try:
                    collection.insert(data_list)
                    print('ok')
                except:
                    print(f'{kw}遺漏')
                data_list = []
                break

        
#多線程執行            
kws1 = [kws[0:30]]
kws2 = [kws[30:60]]
kws3 = [kws[60:90]]
kws4 = [kws[90:120]]
kws5 = [kws[120:164]]

t_list = list() #定議線程

t1 = threading.Thread(target=request_target, args=(kws1))
t_list.append(t1)
t2 = threading.Thread(target=request_target, args=(kws2))
t_list.append(t2)
t3 = threading.Thread(target=request_target, args=(kws3))
t_list.append(t3)
t4 = threading.Thread(target=request_target, args=(kws4))
t_list.append(t4)
t5 = threading.Thread(target=request_target, args=(kws5))
t_list.append(t5)

for t in t_list:
    t.start()

for t in t_list:
    t.join()    
        
client.close()
server.stop()

ADULT INCONTINENCE
CHEESE
EGGS
ICE CREAM/MILK/SHERBTS
PIES
711
ok
IMPORTED WINE
149
ok
PNT BTR/JELLY/JAMS
110
ok
ELECTRICAL SUPPPLIES
34
ok
ANALGESICS
24
ok
FAMILY PLANNING
838
ok
CHEESES
1248
ok
INFANT CARE PRODUCTS
112
ok
POPCORN
42
ok
FD WRAPS/BAGS/TRSH BG
2
ok
PORK
18
ok
ANTACIDS
570
ok
CHICKEN
313
ok
FEMININE HYGIENE
435
139
ok
INFANT FORMULA
ok
PREPARED FOOD
80
ok
APPLES
687
ok
CHICKEN/POULTRY
321
ok
FILM AND CAMERA PRODUCTS
406
ok
AUDIO/VIDEO PRODUCTS
136
ok
INSECTICIDES
15
ok
AUTOMOTIVE PRODUCTS
1248
ok
PREPARED/PKGD FOODS
61
ok
FIREWORKS
614
ok
CHRISTMAS  SEASONAL
99
ok
BABY FOODS
71
ok
KITCHEN GADGETS
32
ok
LAUNDRY ADDITIVES
28
ok
FIRST AID PRODUCTS
141
ok
FITNESS&DIET
1248
0
FITNESS&DIET遺漏
FLORAL- HARD GOODS
ok
PROD SUPPLIES
1248
ok
CIGARETTES
62
ok
LAUNDRY DETERGENTS
699
ok
BABY HBC
43
ok
FLOUR & MEALS
0
PROD SUPPLIES遺漏
PWDR/CRYSTL DRNK MX
60
ok
FLUID MILK PRODUCTS
19
ok
CIGARS
335
ok
LAXATIVES
109
ok
BACON
447
ok
FROZEN
0
FROZEN遺漏
FROZEN MEAT
2
ok
REFRGRATD DOUGH PRODUCTS


2021-07-18 00:47:01,335| ERROR   | Could not establish connection from local ('127.0.0.1', 65441) to remote ('127.0.0.1', 27017) side of the tunnel: 


ok


In [ ]:
['FITNESS&DIET' , 'PROD SUPPLIES' , 'FROZEN' , 'FRZN JCE CONC/DRNKS' , 'SALD DRSNG/SNDWCH SPRD' , 'GIFT & FRUIT BASKETS' , 'PHARMACY'
,'TOYS']

### 變成DataFrame

In [15]:
df = pd.DataFrame({"commodity_SUB_COMMODITY":kw,"commodity_name":commodity_name,'commodity_images':commodity_images,'commodity_price':commodity_price,'commodity_url':commodity_url,'commodity_description':commodity_description
,'commodity_total_reviews':commodity_total_reviews ,'commodity_descriptio':commodity_descriptio,'commodity_average_rating':commodity_average_rating })


In [16]:
df

,commodity_SUB_COMMODITY,commodity_name,commodity_images,commodity_price,commodity_url,commodity_description,commodity_total_reviews,commodity_descriptio,commodity_average_rating
0,STATIONERY & SCHOOL SUPPLIES,Shredded Mexican-Style Cheese - 8oz - Good &#3...,https://target.scene7.com/is/image/Target/GUES...,$1.99,https://www.target.com/p/shredded-mexican-styl...,Whether you're putting together a condiments b...,56,"[Contains: Milk, Form: Shredded, State of Read...",4.23
1,STATIONERY & SCHOOL SUPPLIES,Shredded Mozzarella Cheese - 8oz - Good & Gath...,https://target.scene7.com/is/image/Target/GUES...,$1.99,https://www.target.com/p/shredded-mozzarella-c...,Maybe you're sprinkling cheese over a personal...,53,"[Contains: Milk, Form: Shredded, State of Read...",3.19
2,STATIONERY & SCHOOL SUPPLIES,Colby Jack Cheese - 8oz - Good & Gather&#8482;,https://target.scene7.com/is/image/Target/GUES...,$1.99,https://www.target.com/p/colby-jack-cheese-8oz...,For a versatile cheese that's as tasty on its ...,10,"[Contains: Milk, Form: Brick, State of Readine...",4.60
3,STATIONERY & SCHOOL SUPPLIES,Finely Shredded Cheddar Jack Cheese - 8oz - Go...,https://target.scene7.com/is/image/Target/GUES...,$1.99,https://www.target.com/p/finely-shredded-chedd...,"Give your signature chili, homemade broccoli-c...",7,"[Contains: Milk, Form: Shredded, State of Read...",3.43
4,STATIONERY & SCHOOL SUPPLIES,Mild Cheddar Deli Sliced Cheese - 8oz/12 slice...,https://target.scene7.com/is/image/Target/GUES...,$1.99,https://www.target.com/p/mild-cheddar-deli-sli...,Add a finishing touch to your sandwich with th...,29,"[Contains: Milk, Form: Sliced, State of Readin...",3.79
...,...,...,...,...,...,...,...,...,...
1435,STATIONERY & SCHOOL SUPPLIES,"Staples 4"" x 6"" Line Ruled White Index Cards 5...",https://target.scene7.com/is/image/Target/GUES...,$12.05,https://www.target.com/p/staples-4-x-6-line-ru...,Keep organized notes with this 500-count pack ...,0,"[Number of Pieces: 500, Dimensions (Overall): ...",0.00
1436,STATIONERY & SCHOOL SUPPLIES,JAM Paper Smooth Formal Notecards White Panel ...,https://target.scene7.com/is/image/Target/GUES...,$28.55,https://www.target.com/p/jam-paper-smooth-form...,JAM Paper A2 Size white panel base blank fold ...,0,"[Number of Pieces: 100, Dimensions (Overall): ...",0.00
1437,STATIONERY & SCHOOL SUPPLIES,Best Paper Greetings Pack of 100 Place Cards -...,https://target.scene7.com/is/image/Target/GUES...,$11.99,https://www.target.com/p/best-paper-greetings-...,Package Includes\n100 pcs Place Paper Greeting...,0,"[Number of Pieces: 100, Dimensions (Overall): ...",0.00
1438,STATIONERY & SCHOOL SUPPLIES,My Little Pony 16ct Giant Sticker Valentines,https://target.scene7.com/is/image/Target/GUES...,$3.50,https://www.target.com/p/my-little-pony-16ct-g...,NaN,4,"[Number of Pieces: 16, Dimensions (Overall): 9...",4.25


In [ ]:
df.isna().sum()

commodity_name                0
commodity_images              0
commodity_price               0
commodity_url                 0
commodity_description        43
commodity_average_rating      0
commodity_RatingValue1        0
commodity_RatingValue2        0
commodity_RatingValue3        0
commodity_RatingValue4        0
commodity_RatingValue5        0
commodity_Value_score       214
commodity_Quality_score     207
commodity_Taste_score       209
commodity_total_reviews       0
commodity_highlights         60
commodity_descriptio          0
dtype: int64

In [ ]:
df.sort_values(by=['commodity_name'])  #會有相同的commodity_name	，但某些欄位不一樣

,commodity_name,commodity_images,commodity_price,commodity_url,commodity_description,commodity_average_rating,commodity_RatingValue1,commodity_RatingValue2,commodity_RatingValue3,commodity_RatingValue4,commodity_RatingValue5,commodity_Value_score,commodity_Quality_score,commodity_Taste_score,commodity_total_reviews,commodity_highlights,commodity_descriptio
492,1% Milkfat Small Curd Cottage Cheese - 16oz - ...,https://target.scene7.com/is/image/Target/GUES...,$1.99,https://www.target.com/p/1-milkfat-small-curd-...,Give your morning a flavorful start or power t...,3.00,3,0,0,0,3,3.50,2.33,3.00,6,"[Small-curd cottage cheese with 1% milk fat, M...","[Contains: Milk, May Contain: Soy, Tree Nuts, ..."
227,1% Milkfat Small Curd Cottage Cheese - 24oz - ...,https://target.scene7.com/is/image/Target/GUES...,$2.99,https://www.target.com/p/1-milkfat-small-curd-...,Whether you're looking for a quick breakfast o...,2.96,11,2,2,1,11,2.92,2.38,2.38,27,"[Contains 24oz of low-fat cottage cheese, Made...","[Contains: Milk, May Contain: Soy, Tree Nuts, ..."
286,4% Milkfat Small Curd Cottage Cheese - 24oz - ...,https://target.scene7.com/is/image/Target/GUES...,$2.99,https://www.target.com/p/4-milkfat-small-curd-...,With a container of 4% Milk Fat Small Curd Cot...,3.39,8,4,0,1,8,3.33,3.21,3.26,28,"[Small-curd cottage cheese with 4% milk fat, M...","[Contains: Milk, May Contain: Soy, Tree Nuts, ..."
803,8 Myles Frozen BBQ Mac and Cheese - 8oz,https://target.scene7.com/is/image/Target/GUES...,$4.49,https://www.target.com/p/8-myles-frozen-bbq-ma...,Whole grain elbow pasta tossed in a creamy whi...,5.00,0,0,0,0,0,5.00,5.00,5.00,3,"[Smokey BBQ Sauce, All real ingredients, Whole...","[Contains: Eggs, Milk, Wheat, Features: No Art..."
763,8 Myles Frozen BBQ Mac and Cheese - 8oz,https://target.scene7.com/is/image/Target/GUES...,$4.49,https://www.target.com/p/8-myles-frozen-bbq-ma...,Whole grain elbow pasta tossed in a creamy whi...,5.00,0,0,0,0,0,5.00,5.00,5.00,3,"[Smokey BBQ Sauce, All real ingredients, Whole...","[Contains: Eggs, Milk, Wheat, Features: No Art..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,Whisps Parmesan &#38; Asiago and Pepper Jack C...,https://target.scene7.com/is/image/Target/GUES...,$7.49,https://www.target.com/p/whisps-parmesan-38-as...,NaN,5.00,0,0,0,0,0,NaN,NaN,NaN,1,NaN,"[Contains: Milk, Form: Pieces, State of Readin..."
85,Whisps Parmesan Cheese Crisps - 2.12oz,https://target.scene7.com/is/image/Target/GUES...,$3.99,https://www.target.com/p/whisps-parmesan-chees...,Whisps are made from just one delicious ingred...,4.17,9,2,8,7,9,3.38,4.35,4.00,78,[100% Parmesan Cheese - the only artisanal cop...,"[Contains: Milk, Dietary Needs: Gluten Free, F..."
176,Whisps Tangy Ranch Cheese Crisps - 2.12oz,https://target.scene7.com/is/image/Target/GUES...,$3.69,https://www.target.com/p/whisps-tangy-ranch-ch...,A cheddar-for-you Ranch! What happens when we ...,3.55,2,2,0,2,2,2.67,5.00,3.00,11,"[10g of Protein per Serving, 3g Carbs per Serv...","[Contains: Milk, Form: Pieces, State of Readin..."
686,White Cheddar Mac and Cheese - 20oz - Good & G...,https://target.scene7.com/is/image/Target/GUES...,$3.99,https://www.target.com/p/white-cheddar-mac-and...,Take mac and cheese night up a notch with Whit...,4.68,1,1,0,3,1,4.70,4.85,4.57,31,[Mac and cheese made with whole milk and butte...,"[Contains: Eggs, Milk, Wheat, State of Readine..."
